### CIFO Tetris Block Puzzle

Notebook for the development of a working tetris block puzzle representation. The output should be a function or a library that is able to produce pieces, grids, whatever.

Fred and I developed a conceptual construction for the problem.
First we need a programatical way to represent a solution in the grid.
For that, we can define a list with strings containing two elements:

Individual: `[<Letter><Number>]`, 
where letter represents the piece, and the number represents one of the possible 4 rotations of the piece. 

Given any of these elements, a 2d array representation of it can be built. For example, a square block would occupy the four coordinates `(i,j),(i+1,j),(i,j+1), and (i+1,j+1)`. 

"But how do we know what i and j will be?" This is answered by our proposed approach to filling the grid.
Given the specified configuration for each combination of `<Letter><Number>`, the approach is for the program to take each element of the string sequentially, and place it according to its defined configuration in the first available space in the grid for it. 

Consider an array representation of such a grid, where 0s represent empty spots, and 1 occupied spots. 
The program will look for the first instance of a configuration of zeroes that can match the shape, and place them there. For the square example above, the program looks for the first zero in the grid, found at (0,0). This will be the i,j for the square. It then searches the given spots, checking for zeroes. If the shape can be placed, then the 0s become ones and the program moves to the next piece. If no configuration of matching 0s can be found, the program skips the piece, until it reaches the end of the individual string.

A visualization of this algorithm can be found below:
<br>
<img src="Image1.png" width="550" title="Freds Guide to Teris Block Puzzle Program, Life, and everything">



In [1]:
# Imports.
import numpy as np

In [57]:
## Let's begin.

# Dict of pieces and the coordinates that they will encode.
# A piece will be composed by a number of squares, denoted by that same number of vectors that represents the coordinates of squares 
# that the piece occupies relative to its first square. 
# e.g. an I piece will occupy four rows of the same column in a matrix.

t_dict = {
    "I" : [(0,0),(1,0),(2,0),(3,0)],
    "L" : [(0,0),(1,0),(2,0),(2,1)],
    "J" : [(0,0),(1,0),(2,0),(2,-1)],
    "T" : [(0,0),(1,0),(1,-1),(1,1)],
    "S" : [(0,0),(1,0),(1,-1),(0,1)],
    "Z" : [(0,0),(1,0),(1,1),(0,-1)],
    "O" : [(0,0),(1,0),(0,1),(1,1)]
    
    
}


In [75]:
# Here is the function that will do all the work.

def tetrimino_fitter(piece="I",rotation=1,grid=None):
    """
    Piece is the code for the piece to try to fit.
    Rotation is an integer representing one of four possible rotations for the piece.
    Size is the size of the grid, given as a tuple.
    Grid expects an array. If none exists it will create one for us.
    Returns the grid with the "fitted" pieces.
    """
    # Transform the corresponding piece into its matrix representation.
    # Consider rotation and how that impacts the piece, and apply the corresponding transformation here.
    configuration = rotate_piece(configuration=t_dict[piece],rotation=rotation)
    
    # Initialize the grid with a shape of 4,4 if None exists, or if it is not a numpy array.
    if type(grid)!=np.ndarray:
        grid=np.zeros((4,4))
    
    # Identify spots that can be filled. 
    
    free_space_array = np.column_stack(np.where(grid==0)) # this can be used to find the coordinates of all the zeroes, and have them stored.

    for coord in free_space_array:
        fit_array = []
        print("First zero in sequence found at {}".format(tuple(coord)))

        for t in configuration:
            break_flag = False
            c = tuple(np.add(tuple(coord),tuple(t))) # c is the coordinate to be checked. It can be found by adding the vectors in configurations to the coordinates where the zero was detected.
            
            # Check if the coordinate is valid for the grid.
            if c < (0,0) or grid.shape < tuple(np.add(c,(1,1))):
                break_flag = True
                print("Configuration out of bounds, checking next zero.")
                break
            
            elif grid[c]==0:
                print("Empty space at coordinates {}".format(c))
                fit_array.append(c)
                continue # If the space is open, check the others. Else, break that loop and check for the next zero.
            else:
                break_flag = True
                print("No next zeroes in the sequence, to the next one")
                break
                
        if break_flag == False: ## if the for loop ended by finding all the zeroes, then the piece fits.
            print(fit_array)
            for _ in fit_array:
                grid[_] = 1 # These spots are now occupied
            break # from the other loop
        else:
            continue # If the configuration isn't valid, go into the next zero found.

    
    
    return grid

In [76]:
def rotate_piece(configuration,rotation):
    """
    Transforms a given piece configuration (given by t_dict, or a tensor of coordinates) and returns that transformation. 
    Piece is the tensor that represents the piece configuration.
    Rotation is the desired rotation for the piece. As it is defined, 2 is 90 degree rotation clockwise, 3 is 180, and 4 is 270. 
    """
    # If rotation is a string, transform it into integer...
    rotation = int(rotation)
    
    # Numpy operations makes these transformations really easy.
    c = np.array(configuration)
    
    if rotation == 2:
        c = np.flip(c,axis=1)
        c[:,1] *= -1
    elif rotation == 3:
        c *= -1
    elif rotation == 4:
        c = np.flip(c,axis=1)
        c[:,0] *= -1
    else:
        pass
    
    return c.tolist()

In [77]:
tetrimino_fitter()

First zero in sequence found at (0, 0)
Empty space at coordinates (0, 0)
Empty space at coordinates (1, 0)
Empty space at coordinates (2, 0)
Empty space at coordinates (3, 0)
[(0, 0), (1, 0), (2, 0), (3, 0)]


array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [78]:
# One more test cell.
grid = np.zeros((4,4))
for piece in list(t_dict.keys()):
    grid = tetrimino_fitter(piece=piece,grid=grid)
    print(grid)

First zero in sequence found at (0, 0)
Empty space at coordinates (0, 0)
Empty space at coordinates (1, 0)
Empty space at coordinates (2, 0)
Empty space at coordinates (3, 0)
[(0, 0), (1, 0), (2, 0), (3, 0)]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
First zero in sequence found at (0, 1)
Empty space at coordinates (0, 1)
Empty space at coordinates (1, 1)
Empty space at coordinates (2, 1)
Empty space at coordinates (2, 2)
[(0, 1), (1, 1), (2, 1), (2, 2)]
[[1. 1. 0. 0.]
 [1. 1. 0. 0.]
 [1. 1. 1. 0.]
 [1. 0. 0. 0.]]
First zero in sequence found at (0, 2)
Empty space at coordinates (0, 2)
Empty space at coordinates (1, 2)
No next zeroes in the sequence, to the next one
First zero in sequence found at (0, 3)
Empty space at coordinates (0, 3)
Empty space at coordinates (1, 3)
Empty space at coordinates (2, 3)
No next zeroes in the sequence, to the next one
First zero in sequence found at (1, 2)
Empty space at coordinates (1, 2)
No next zeroes in the sequence, to the next o

In [ ]:
# Visual Output for a Grid, and pieces that fill it.

In [45]:
## tiago is working on this cell and coming up with really weird ideas to try out
## (he-s really going into numpy.)
## You can safely ignore this cell as it's just a lot of tests being done.

# Initialize the grid.
size = (4,4)
m = np.array([0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,1]).reshape((4,4))

# Identify spots that can be filled. 

# Verifies if a configuration of empty spaces for the piece exists.
# if 1 in [m[tuple(np.add((0,0),coord))] for coord in t_dict["L"]]
print(m)
free_space_array = np.column_stack(np.where(m==0)) # this can be used to find the coordinates of all the zeroes, and have them stored.

for coord in free_space_array:
    fit_array = []
    print("First zero in sequence found at {}".format(tuple(coord)))
    
    for t in t_dict["L"]:
        break_flag = False
        c = tuple(np.add(tuple(coord),t))
        if m[c]==0:
            print("Empty space at coordinates {}".format(c))
            fit_array.append(c)
            continue # If the space is open, check the others. Else, break that loop and check for the next zero.
        else:
            break_flag = True
            print("No next zeroes in the sequence, to the next one")
            break
    if break_flag == False: ## if the for loop ended by finding all the zeroes, then the piece can fit.
        print(fit_array)
        for _ in fit_array:
            m[_] = 1 # These spots are now occupied
        break # from the other loop
    else:
        continue # If the configuration isn't valid, go into the next zero found.

print(m)
    
        
# No matter what I magic up, I can't seem to find a way to find out the correct configuration of space in the grid without doing a sequential search method.

[[0 0 0 0]
 [0 1 1 1]
 [0 0 0 0]
 [0 1 1 1]]
First zero in sequence found at (0, 0)
Empty space at coordinates (0, 0)
Empty space at coordinates (1, 0)
Empty space at coordinates (2, 0)
Empty space at coordinates (2, 1)
[(0, 0), (1, 0), (2, 0), (2, 1)]
[[1 0 0 0]
 [1 1 1 1]
 [1 1 0 0]
 [0 1 1 1]]
